# Acquisition Exercises

The end result of this exercise should be a file named acquire.py.

In [1]:
import pandas as pd
import numpy as np
import requests

### 1. Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items as we did in the lesson, create a dataframe named items that has all of the data for items.

In [2]:
response = requests.get('https://python.zgulde.net/api/v1/items')
response

<Response [200]>

In [3]:
data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [4]:
data['payload'].keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [5]:
items = pd.DataFrame(data['payload']['items'])
items.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [6]:
base_url = 'https://python.zgulde.net'


In [7]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

items = pd.concat([items, pd.DataFrame(data['payload']['items'])])

max_page: 3
next_page: /api/v1/items?page=3


In [8]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

items = pd.concat([items, pd.DataFrame(data['payload']['items'])])

max_page: 3
next_page: None



### 2. Do the same thing, but for stores (https://python.zgulde.net/api/v1/stores)

In [9]:
response = requests.get('https://python.zgulde.net/api/v1/stores')
data = response.json()
stores = pd.DataFrame(data['payload']['stores'])
stores.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


### 3. Extract the data for sales (https://python.zgulde.net/api/v1/sales). There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [10]:
response = requests.get('https://python.zgulde.net/api/v1/sales')
data = response.json()
max_page = data['payload']['max_page']
current_page = 0
sales = pd.DataFrame()

while current_page != (max_page - 1):
    response = requests.get(base_url + data['payload']['next_page'])
    data = response.json()
    sales = pd.concat([sales, pd.DataFrame(data['payload']['sales'])])
    current_page += 1
    if current_page > max_page:
        break

### 4. Save the data in your files to local csv files so that it will be faster to access in the future.

In [11]:
sales.to_csv('sales.csv')
stores.to_csv('stores.csv')
items.to_csv('items.csv')

### 5. Combine the data from your three separate dataframes into one large dataframe.


In [12]:
sales.rename(columns = {'store':'store_id', 'item':'item_id'},inplace = True)

In [13]:
sales.shape

(908000, 5)

In [14]:
df = pd.merge(sales, stores, on = 'store_id', how = 'left')
df = pd.merge(df, items, on = 'item_id', how = 'left')

In [15]:
df.head()

,item_id,sale_amount,sale_date,sale_id,store_id,store_address,store_city,store_state,store_zipcode,item_brand,item_name,item_price,item_upc12,item_upc14
0,1,33.0,"Sat, 10 Sep 2016 00:00:00 GMT",5001,3,2118 Fredericksburg Rdj,San Antonio,TX,78201,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,27.0,"Sun, 11 Sep 2016 00:00:00 GMT",5002,3,2118 Fredericksburg Rdj,San Antonio,TX,78201,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
2,1,26.0,"Mon, 12 Sep 2016 00:00:00 GMT",5003,3,2118 Fredericksburg Rdj,San Antonio,TX,78201,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
3,1,22.0,"Tue, 13 Sep 2016 00:00:00 GMT",5004,3,2118 Fredericksburg Rdj,San Antonio,TX,78201,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
4,1,25.0,"Wed, 14 Sep 2016 00:00:00 GMT",5005,3,2118 Fredericksburg Rdj,San Antonio,TX,78201,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013


In [16]:
df.shape

(908000, 14)

### 6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv


In [17]:
df = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')

### 7. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.

In [18]:
import acquire

In [19]:
items = acquire.get_items()
stores = acquire.get_stores()
sales = acquire.get_sales()
opsd_germany = acquire.get_opsd_germany()

In [20]:
df = acquire.get_commerce(sales, items, stores)